In [5]:
# one off operation
!pip install -q duckdb

StatementMeta(, 9da46e06-8556-4440-8088-70469319de03, 7, Finished, Available, Finished)

Register an App if running outside Fabric to get a token
https://fivetran.com/docs/destinations/onelake/setup-guide

In [6]:
%%time
import duckdb
def duckdb_attach_lakehouse(workspace,LH):
    ########## get token
    try:
      token = notebookutils.credentials.getToken('storage')
    except:
        !pip install -q azure-identity
        from   azure.identity   import ClientSecretCredential
        import configparser
        config = configparser.ConfigParser()
        config.read("C:/KV/variable.ini")
        credential = ClientSecretCredential(
                        client_id     =config.get("myvars", "appId"),
                        client_secret =config.get("myvars", "secret") , 
                        tenant_id     =config.get("myvars", "tenantId")
                        )
        token =       credential.get_token("https://storage.azure.com/.default").token
    duckdb.sql(f""" CREATE or replace SECRET secret4 ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}') """)
    ########## check if the lakehouse has a schema 
    try:
        duckdb.sql(f""" SELECT  * FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/dbo/*") """).df()['file'].tolist()
        withschema = 'yes'
    except:
        withschema = 'no'
    sql_schema = set()
    sql_statements = set()
    if withschema == 'yes':
        list_tables = duckdb.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/*/_delta_log/00000000000000000000.json") """).df()['tables'].tolist()
        for table_path in list_tables:
            parts = table_path.strip("/").split("/")
            schema = parts[-2]
            table = parts[-1]
            sql_schema.add(f"CREATE SCHEMA IF NOT EXISTS {schema};")
            sql_statements.add(f"""CREATE OR REPLACE VIEW {schema}.{table} AS SELECT * 
                                FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{schema}/{table}');""")
        duckdb.sql(" ".join(sql_schema))
    else:
        list_tables = duckdb.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/_delta_log/00000000000000000000.json") """).df()['tables'].tolist()
        for table_path in list_tables:
            parts = table_path.strip("/").split("/")
            table = parts[-1]
            sql_statements.add(f"""CREATE OR REPLACE VIEW {table} AS SELECT * 
                                FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{table}');""")
    duckdb.sql(" ".join(sql_statements))
    duckdb.sql("SHOW ALL TABLES").show(max_width=100)

StatementMeta(, 9da46e06-8556-4440-8088-70469319de03, 8, Finished, Available, Finished)

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 13.8 µs


In [7]:
duckdb_attach_lakehouse("sqlengines","ETL")

StatementMeta(, 9da46e06-8556-4440-8088-70469319de03, 9, Finished, Available, Finished)

┌──────────┬─────────┬──────────────────┬──────────────────────┬───────────────────────┬───────────┐
│ database │ schema  │       name       │     column_names     │     column_types      │ temporary │
│ varchar  │ varchar │     varchar      │      varchar[]       │       varchar[]       │  boolean  │
├──────────┼─────────┼──────────────────┼──────────────────────┼───────────────────────┼───────────┤
│ memory   │ T1      │ chdb             │ [UNIT, VERSION, SE…  │ [VARCHAR, DOUBLE, I…  │ false     │
│ memory   │ T1      │ duckdb           │ [UNIT, DUID, filen…  │ [VARCHAR, VARCHAR, …  │ false     │
│ memory   │ T1      │ duckdbparquet    │ [UNIT, DUID, filen…  │ [VARCHAR, VARCHAR, …  │ false     │
│ memory   │ T1      │ pandas           │ [UNIT, version, SE…  │ [VARCHAR, BIGINT, T…  │ false     │
│ memory   │ T1      │ pyarrow          │ [year, UNIT, VERSI…  │ [BIGINT, VARCHAR, D…  │ false     │
│ memory   │ T10     │ duckdb           │ [UNIT, DUID, filen…  │ [VARCHAR, VARCHAR, …  │ fa